In [1]:
import pandas as pd
import numpy as np
import scipy as sp
# Open graphs in new cells in the page rather than in a separate window
%matplotlib inline 
import matplotlib.pyplot as plt 
plt.rcParams["figure.figsize"] = (15, 5)  # set a default figure size


In [5]:
df = pd.read_csv('answers.csv')

In [6]:
# Look at the first 3 rows
df[:3]

,Unnamed: 0,qid,i,qs,qt,tags,qvc,qac,aid,j,as,at
0,1,563355,62701.0,0,1235000081,"php,error,gd,image-processing",220,2,563372,67183.0,2,1235000501
1,2,563355,62701.0,0,1235000081,"php,error,gd,image-processing",220,2,563374,66554.0,0,1235000551
2,3,563356,15842.0,10,1235000140,"lisp,scheme,subjective,clojure",1047,16,563358,15842.0,3,1235000177


Columns values,
qid: Unique question id

i: User id of questioner

qs: Score of the question

qt: Time of the question (in epoch time)

tags: a comma-separated list of the tags associated with the question. Examples of tags are ``html'', ``R'', ``mysql'', ``python'', and so on; often between two and six tags are used on each question.

qvc: Number of views of this question (at the time of the datadump)

qac: Number of answers for this question (at the time of the datadump)

aid: Unique answer id

j: User id of answerer

as: Score of the answer

at: Time of the answer

In [17]:
df['tags'][:10]

0     php,error,gd,image-processing
1     php,error,gd,image-processing
2    lisp,scheme,subjective,clojure
3    lisp,scheme,subjective,clojure
4    lisp,scheme,subjective,clojure
5    lisp,scheme,subjective,clojure
6    lisp,scheme,subjective,clojure
7    lisp,scheme,subjective,clojure
8    lisp,scheme,subjective,clojure
9    lisp,scheme,subjective,clojure
Name: tags, dtype: object

In [22]:
features =  set()
def feature_selection(data):
    for rows in data:
        
        for feat in  rows.split(','):
            features.add(feat)
                        

In [23]:
feature_selection(df['tags']) # Get list of all the unique tags in features
print(len(features))

0
14248


In [ ]:
#for e in features:
#    print(e)
#    if e is 'lisp':
#        break

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(norm='l2', vocabulary=list(features))

In [27]:
training_matrix =  tf.fit_transform(df['tags'])
training_feature_names = tf.get_feature_names() 

In [28]:
print(training_feature_names)

['pdf-generation', 'icpc', 'mtasc', 'personal-development', 'classic-asp', 'trailing-character', 'lookandfeel', 'small-projects', 'x11', 'database-structure', 'sitetemplate', 'zenity', 'random-string', 'codecave', 'bubble', 'netapi32', 'jinternalframe', 'jquery-grid', 'software-process', 'print-webpage', 'stateful-session-bean', 'printing-web-page', 'crm', 'fixed-length-record', 'desktopheap', 'filegetcontent', 'browser-automation', 'monomorphism', 'spec#', 'exporting', 'real-world', 'pygame', 'ûnet4û1', 'local-reference', 'works-on-my-machine', 'zlib', 'web-hosting', 'serious', 'linear-algebra', '2û5û2', 'cname', 'benefits', 'sqlcompare', 'hql', 'mru', 'maestro', 'squirrel-sql', 'oracle10g', 'gps', 'thoughtworks', 'remote-display', 'businessobjects', 'uipi', 'sortables', 'activemq', 'characters', 'sql-variant', 'intersection', 'mdi', 'project', 'html-email', 'quote', 'multiplicative-order', 'modeling', 'director', 'jp2', 'economy', 'bug-classification', 'regsvr32', 'file-handling', 'v

In [29]:
training_matrix[:2]

<2x14248 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>